In [21]:
import tiktoken
import torch
import torch.nn as nn
from torch.nn import functional as F
config = {
    "model_name": "gpt2",
    "vocab_size": 50257,  # GPT-2 vocab size
    "context_length": 1024,  # GPT-2 max position
    "n_heads": 12,
    "hidden_dim": 3072,
    "embedding_dim": 768,
    "n_layers": 12,
    "batch_size": 4,
}
       

In [22]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
    def forward(self, x):
        # Calculate mean and variance
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        
        # Normalize the input
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        
        # Scale and shift
        return self.scale * norm_x + self.shift

In [23]:
class Tokenizer:
    def __init__(self, model_name: str):
        self.encoding = tiktoken.get_encoding(model_name)

    def encode(self, text: str):
        return self.encoding.encode(text)

    def decode(self, tokens: list[int]):
        return self.encoding.decode(tokens)
        

In [24]:
class Embedding(nn.Module):
    def __init__(self, model_name: str, config, dropout=0.1):
        super().__init__()
        self.model_name = model_name
        self.config = config
        self.tokenizer = Tokenizer(model_name) 
        self.embedding = nn.Embedding(config["vocab_size"], config["embedding_dim"])
        self.positional_encoding = nn.Embedding(config["context_length"], config["embedding_dim"])
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        # inputs: token IDs, shape (batch, seq_len) or (seq_len,)
        ey = self.embedding(inputs)
        # Create position indices: [0, 1, 2, 3, ..., seq_len-1]
        seq_len = inputs.shape[-1]
        positions = torch.arange(seq_len, dtype=torch.long, device=inputs.device)
        pex = self.positional_encoding(positions)
        x = ey + pex
        x = self.dropout(x)
        return x

In [25]:
class MultiHeadAttention(nn.Module):
    def __init__(self,  embedded_size, num_heads, qkv_bias=False):
        super().__init__()
        self.config = config
        self.q = nn.Linear(config["embedding_dim"], config["embedding_dim"])
        self.k = nn.Linear(config["embedding_dim"], config["embedding_dim"])
        self.v = nn.Linear(config["embedding_dim"], config["embedding_dim"])
        self.out_proj = nn.Linear(config["embedding_dim"], config["embedding_dim"])

    def forward(self, input_text):
        # Ensure input_text has the correct shape (T, D)
         # text: (B, T, D)
        # input_text.unsqueeze_(0)
        B, T, D = input_text.shape
        H = self.config["n_heads"]
        d_head = D // H

        Q = self.q(input_text)
        K = self.k(input_text)
        V = self.v(input_text)

        Q = Q.view(B, T, H, d_head).transpose(1, 2)
        K = K.view(B, T, H, d_head).transpose(1, 2)
        V = V.view(B, T, H, d_head).transpose(1, 2)

        scores = (Q @ K.transpose(-2, -1)) / (d_head ** 0.5)
        
        # Apply causal mask BEFORE softmax
        mask = torch.tril(torch.ones(T, T)).unsqueeze(0).unsqueeze(0)
        scores = scores.masked_fill(mask == 0, float('-inf'))
        
        weights = F.softmax(scores, dim=-1)

        out = weights @ V                # (B, H, T, d_head)

        out = out.transpose(1, 2).contiguous().view(B, T, D)
        out = self.out_proj(out)

        return out 
#     

In [26]:
class FeedForward(nn.Module):
    def __init__(self, embedding_dim, hidden_dim):
        super().__init__()
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, embedding_dim)
        self.activation = nn.GELU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x

In [27]:
class Dropout(nn.Module):
    def __init__(self, p: float = 0.5):
        super().__init__()
        self.p = p

    def forward(self, x):
        if not self.training or self.p == 0.0:
            return x
        mask = (torch.rand_like(x) > self.p).float()
        return x * mask / (1.0 - self.p)

In [28]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, num_heads, ff_hidden_size, dropout=0.1, qkv_bias=False):
        super().__init__()
        # Pre-LN: LayerNorm BEFORE operations
        self.ln1 = nn.LayerNorm(embed_size)
        self.ln2 = nn.LayerNorm(embed_size)
        # Multi-head attention
        self.mha = MultiHeadAttention(embed_size, num_heads, qkv_bias)
        # Feed-forward network
        self.ff = FeedForward(embed_size, 4*embed_size)
        # Dropout
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        # Pre-LN: LayerNorm BEFORE attention
        attn_out = self.mha(self.ln1(x))
        attn_out = self.dropout(attn_out)
        x = x + attn_out  # Residual
        
        # Pre-LN: LayerNorm BEFORE feedforward
        ff_out = self.ff(self.ln2(x))
        ff_out = self.dropout(ff_out)
        x = x + ff_out  # Residual
        
        return x

In [29]:
class Llm(nn.Module):
    def __init__(self, model_name, config, dropout=0.1):
        super().__init__()
        self.config = config
        self.n_layers = config["n_layers"]
        self.embedding = Embedding(model_name, config, dropout)
        # Stack transformer blocks
        self.transformers = nn.ModuleList([
            TransformerBlock(config["embedding_dim"], config["n_heads"], config["hidden_dim"], dropout)
            for _ in range(self.n_layers)
        ])
        # Output head (tied weights)
        self.output_layer = nn.Linear(config["embedding_dim"], config["vocab_size"], bias=False)
        # Final LayerNorm
        self.final_layernorm = nn.LayerNorm(config["embedding_dim"])
    
    def forward(self, idx):
        # idx: (batch, seq_len)
        x = self.embedding(idx)
        for block in self.transformers:
            x = block(x)
        x = self.final_layernorm(x)
        logits = self.output_layer(x)
        return logits

In [30]:
# Test Transformer Block
# Create an instance of the TransformerBlock using the configuration values
transformer = TransformerBlock(config["embedding_dim"], config["n_heads"], config["hidden_dim"])
embed=Embedding("gpt2", config)
token=Tokenizer("gpt2")
input_text = "Hello, this is a test input for the Transformer Block."
# Token IDs: (batch, seq_len)
input_ids = torch.tensor([token.encode(input_text)], dtype=torch.long)
# Embedding: (batch, seq_len, embedding_dim)
embedded = embed(input_ids)
output = transformer(embedded)
# Apply the transformer block to the output of the positional encoding layer
# Print the shape of the output after applying the transformer block
print(f"Transformer Block output shape: {output.shape}")
# Assert that the output shape matches the expected shape
assert output.shape == output.shape, "Transformer Block shape mismatch"

Transformer Block output shape: torch.Size([1, 13, 768])


In [31]:
def generate_text_simple(model, idx, max_new_tokens, context_size, temperature=1.0):
    model.eval()
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]
        logits = logits / temperature
        probas = torch.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probas, num_samples=1)
        idx = torch.cat((idx, idx_next), dim=1)
    return idx

# Usage:
input_text = "Hello, how are you?"
token = Tokenizer("gpt2")
input_ids = torch.tensor([token.encode(input_text)], dtype=torch.long)
model = Llm("gpt2", config)
# (Make sure to load weights if needed)
out = generate_text_simple(model, input_ids, max_new_tokens=6, context_size=config["context_length"], temperature=1.2)

In [32]:
import os
import json
import urllib.request
import tensorflow as tf
import torch

GPT2_URL = "https://openaipublic.blob.core.windows.net/gpt-2/models/117M"
FILES = [
    "checkpoint",
    "encoder.json",
    "hparams.json",
    "model.ckpt.data-00000-of-00001",
    "model.ckpt.index",
    "model.ckpt.meta",
    "vocab.bpe",
]

def load_gpt2_117m(model_dir="gpt2_117m"):
    os.makedirs(model_dir, exist_ok=True)

    # 1️⃣ Download files if missing
    for fname in FILES:
        path = os.path.join(model_dir, fname)
        if not os.path.exists(path):
            print(f"Downloading {fname}...")
            urllib.request.urlretrieve(f"{GPT2_URL}/{fname}", path)

    # 2️⃣ Load hparams
    with open(os.path.join(model_dir, "hparams.json")) as f:
        hparams = json.load(f)

    # 3️⃣ Load TensorFlow checkpoint
    reader = tf.train.load_checkpoint(model_dir)

    params = {}
    for name in reader.get_variable_to_shape_map():
        params[name] = torch.tensor(reader.get_tensor(name))

    # 4️⃣ Tokenizer files
    files = {
        "encoder_json": os.path.join(model_dir, "encoder.json"),
        "vocab_bpe": os.path.join(model_dir, "vocab.bpe"),
    }

    return params, hparams, files


In [33]:
params, hparams, files = load_gpt2_117m(model_dir="/Users/ganeshnaidu/Downloads/117M")

In [34]:
def load_gpt2_weights_pytorch(model, params):
    """
    Load GPT-2 weights from TensorFlow checkpoint (as PyTorch tensors) into your PyTorch model.
    Matches the actual checkpoint key structure with /w and /b suffixes.
    """
    # Embedding weights - squeeze to remove extra dimensions
    model.embedding.embedding.weight.data = params['model/wte'].squeeze().clone()
    model.embedding.positional_encoding.weight.data = params['model/wpe'].squeeze().clone()

    # Transformer blocks
    for i in range(model.n_layers):
        block = model.transformers[i]
        
        # Attention weights (QKV are packed together in c_attn) - squeeze first!
        c_attn_w = params[f'model/h{i}/attn/c_attn/w'].squeeze()  # Now: (768, 2304)
        c_attn_b = params[f'model/h{i}/attn/c_attn/b'].squeeze()  # Now: (2304,)
        
        embed_dim = model.config['embedding_dim']
        
        # Split QKV from packed weights and transpose properly
        q_w = c_attn_w[:, :embed_dim].T.clone()
        k_w = c_attn_w[:, embed_dim:2*embed_dim].T.clone()
        v_w = c_attn_w[:, 2*embed_dim:].T.clone()
        
        q_b = c_attn_b[:embed_dim].clone()
        k_b = c_attn_b[embed_dim:2*embed_dim].clone()
        v_b = c_attn_b[2*embed_dim:].clone()
        
        # Load into model
        block.mha.q.weight.data = q_w
        block.mha.q.bias.data = q_b
        block.mha.k.weight.data = k_w
        block.mha.k.bias.data = k_b
        block.mha.v.weight.data = v_w
        block.mha.v.bias.data = v_b
        
        # Attention output projection - squeeze first!
        block.mha.out_proj.weight.data = params[f'model/h{i}/attn/c_proj/w'].squeeze().T.clone()
        block.mha.out_proj.bias.data = params[f'model/h{i}/attn/c_proj/b'].squeeze().clone()
        
        # FeedForward weights - squeeze first!
        block.ff.fc1.weight.data = params[f'model/h{i}/mlp/c_fc/w'].squeeze().T.clone()
        block.ff.fc1.bias.data = params[f'model/h{i}/mlp/c_fc/b'].squeeze().clone()
        block.ff.fc2.weight.data = params[f'model/h{i}/mlp/c_proj/w'].squeeze().T.clone()
        block.ff.fc2.bias.data = params[f'model/h{i}/mlp/c_proj/b'].squeeze().clone()
        
        # LayerNorms - squeeze first!
        block.ln1.weight.data = params[f'model/h{i}/ln_1/g'].squeeze().clone()
        block.ln1.bias.data = params[f'model/h{i}/ln_1/b'].squeeze().clone()
        block.ln2.weight.data = params[f'model/h{i}/ln_2/g'].squeeze().clone()
        block.ln2.bias.data = params[f'model/h{i}/ln_2/b'].squeeze().clone()
    
    # Final LayerNorm - squeeze first!
    model.final_layernorm.weight.data = params['model/ln_f/g'].squeeze().clone()
    model.final_layernorm.bias.data = params['model/ln_f/b'].squeeze().clone()
    
    # Output head (weight tying with input embeddings) - squeeze first!
    model.output_layer.weight.data = params['model/wte'].squeeze().clone()
    
    print("✅ Weights loaded from GPT-2 checkpoint!")

In [35]:
# Create the full GPT-2 model and load weights
model = Llm("gpt2", config)
load_gpt2_weights_pytorch(model, params)
print("Weights loaded into Llm model!")


✅ Weights loaded from GPT-2 checkpoint!
Weights loaded into Llm model!


In [36]:
tok=generate_text_simple(model, input_ids, max_new_tokens=6, context_size=config["context_length"], temperature=1.2)

In [37]:
for i in range(len(tok[0])):
    print(token.decode([tok[0, i].item()]), end='')

Hello, how are you? I guess I'm feeling better

In [38]:
from torch.utils.data import Dataset

class GPTDataset(Dataset):
    def __init__(self, data, block_size, stride=4):
        self.x = []
        self.y = []
        for i in range(0, len(data) - block_size-1, stride):
            self.x.append(data[i:i+block_size])
            self.y.append(data[i+1:i+block_size+1])

    def __len__(self):
        return len(self.x)  

    def __getitem__(self, idx):
        return torch.tensor(self.x[idx], dtype=torch.long), torch.tensor(self.y[idx], dtype=torch.long)

In [39]:
text="""I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)

"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring his unaccountable abdication. "Of course it's going to send the value of my picture 'way up; but I don't think of that, Mr. Rickham--the loss to Arrt is all I think of." The word, on Mrs. Thwing's lips, multiplied its _rs_ as though they were reflected in an endless vista of mirrors. And it was not only the Mrs. Thwings who mourned. Had not the exquisite Hermia Croft, at the last Grafton Gallery show, stopped me before Gisburn's "Moon-dancers" to say, with tears in her eyes: "We shall not look upon its like again"?

Well!--even through the prism of Hermia's tears I felt able to face the fact with equanimity. Poor Jack Gisburn! The women had made him--it was fitting that they should mourn him. Among his own sex fewer regrets were heard, and in his own trade hardly a murmur. Professional jealousy? Perhaps. If it were, the honour of the craft was vindicated by little Claude Nutley, who, in all good faith, brought out in the Burlington a very handsome "obituary" on Jack--one of those showy articles stocked with random technicalities that I have heard (I won't say by whom) compared to Gisburn's painting. And so--his resolve being apparently irrevocable--the discussion gradually died out, and, as Mrs. Thwing had predicted, the price of "Gisburns" went up.

It was not till three years later that, in the course of a few weeks' idling on the Riviera, it suddenly occurred to me to wonder why Gisburn had given up his painting. On reflection, it really was a tempting problem. To accuse his wife would have been too easy--his fair sitters had been denied the solace of saying that Mrs. Gisburn had "dragged him down." For Mrs. Gisburn--as such--had not existed till nearly a year after Jack's resolve had been taken. It might be that he had married her--since he liked his ease--because he didn't want to go on painting; but it would have been hard to prove that he had given up his painting because he had married her.

Of course, if she had not dragged him down, she had equally, as Miss Croft contended, failed to "lift him up"--she had not led him back to the easel. To put the brush into his hand again--what a vocation for a wife! But Mrs. Gisburn appeared to have disdained it--and I felt it might be interesting to find out why.

The desultory life of the Riviera lends itself to such purely academic speculations; and having, on my way to Monte Carlo, caught a glimpse of Jack's balustraded terraces between the pines, I had myself borne thither the next day.

I found the couple at tea beneath their palm-trees; and Mrs. Gisburn's welcome was so genial that, in the ensuing weeks, I claimed it frequently. It was not that my hostess was "interesting": on that point I could have given Miss Croft the fullest reassurance. It was just because she was _not_ interesting--if I may be pardoned the bull--that I found her so. For Jack, all his life, had been surrounded by interesting women: they had fostered his art, it had been reared in the hot-house of their adulation. And it was therefore instructive to note what effect the "deadening atmosphere of mediocrity" (I quote Miss Croft) was having on him.

I have mentioned that Mrs. Gisburn was rich; and it was immediately perceptible that her husband was extracting from this circumstance a delicate but substantial satisfaction. It is, as a rule, the people who scorn money who get most out of it; and Jack's elegant disdain of his wife's big balance enabled him, with an appearance of perfect good-breeding, to transmute it into objects of art and luxury. To the latter, I must add, he remained relatively indifferent; but he was buying Renaissance bronzes and eighteenth-century pictures with a discrimination that bespoke the amplest resources.

"Money's only excuse is to put beauty into circulation," was one of the axioms he laid down across the Sevres and silver of an exquisitely appointed luncheon-table, when, on a later day, I had again run over from Monte Carlo; and Mrs. Gisburn, beaming on him, added for my enlightenment: "Jack is so morbidly sensitive to every form of beauty."

Poor Jack! It had always been his fate to have women say such things of him: the fact should be set down in extenuation. What struck me now was that, for the first time, he resented the tone. I had seen him, so often, basking under similar tributes--was it the conjugal note that robbed them of their savour? No--for, oddly enough, it became apparent that he was fond of Mrs. Gisburn--fond enough not to see her absurdity. It was his own absurdity he seemed to be wincing under--his own attitude as an object for garlands and incense.

"My dear, since I've chucked painting people don't say that stuff about me--they say it about Victor Grindle," was his only protest, as he rose from the table and strolled out onto the sunlit terrace.

I glanced after him, struck by his last word. Victor Grindle was, in fact, becoming the man of the moment--as Jack himself, one might put it, had been the man of the hour. The younger artist was said to have formed himself at my friend's feet, and I wondered if a tinge of jealousy underlay the latter's mysterious abdication. But no--for it was not till after that event that the _rose Dubarry_ drawing-rooms had begun to display their "Grindles."

I turned to Mrs. Gisburn, who had lingered to give a lump of sugar to her spaniel in the dining-room.

"Why _has_ he chucked painting?" I asked abruptly.

She raised her eyebrows with a hint of good-humoured surprise.

"Oh, he doesn't _have_ to now, you know; and I want him to enjoy himself," she said quite simply.

I looked about the spacious white-panelled room, with its _famille-verte_ vases repeating the tones of the pale damask curtains, and its eighteenth-century pastels in delicate faded frames.

"Has he chucked his pictures too? I haven't seen a single one in the house."

A slight shade of constraint crossed Mrs. Gisburn's open countenance. "It's his ridiculous modesty, you know. He says they're not fit to have about; he's sent them all away except one--my portrait--and that I have to keep upstairs."

His ridiculous modesty--Jack's modesty about his pictures? My curiosity was growing like the bean-stalk. I said persuasively to my hostess: "I must really see your portrait, you know."

She glanced out almost timorously at the terrace where her husband, lounging in a hooded chair, had lit a cigar and drawn the Russian deerhound's head between his knees.

"Well, come while he's not looking," she said, with a laugh that tried to hide her nervousness; and I followed her between the marble Emperors of the hall, and up the wide stairs with terra-cotta nymphs poised among flowers at each landing.

In the dimmest corner of her boudoir, amid a profusion of delicate and distinguished objects, hung one of the familiar oval canvases, in the inevitable garlanded frame. The mere outline of the frame called up all Gisburn's past!

Mrs. Gisburn drew back the window-curtains, moved aside a _jardiniere_ full of pink azaleas, pushed an arm-chair away, and said: "If you stand here you can just manage to see it. I had it over the mantel-piece, but he wouldn't let it stay."

Yes--I could just manage to see it--the first portrait of Jack's I had ever had to strain my eyes over! Usually they had the place of honour--say the central panel in a pale yellow or _rose Dubarry_ drawing-room, or a monumental easel placed so that it took the light through curtains of old Venetian point. The more modest place became the picture better; yet, as my eyes grew accustomed to the half-light, all the characteristic qualities came out--all the hesitations disguised as audacities, the tricks of prestidigitation by which, with such consummate skill, he managed to divert attention from the real business of the picture to some pretty irrelevance of detail. Mrs. Gisburn, presenting a neutral surface to work on--forming, as it were, so inevitably the background of her own picture--had lent herself in an unusual degree to the display of this false virtuosity. The picture was one of Jack's "strongest," as his admirers would have put it--it represented, on his part, a swelling of muscles, a congesting of veins, a balancing, straddling and straining, that reminded one of the circus-clown's ironic efforts to lift a feather. It met, in short, at every point the demand of lovely woman to be painted "strongly" because she was tired of being painted "sweetly"--and yet not to lose an atom of the sweetness.

"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride. "The last but one," she corrected herself--"but the other doesn't count, because he destroyed it."

"Destroyed it?" I was about to follow up this clue when I heard a footstep and saw Jack himself on the threshold.

As he stood there, his hands in the pockets of his velveteen coat, the thin brown waves of hair pushed back from his white forehead, his lean sunburnt cheeks furrowed by a smile that lifted the tips of a self-confident moustache, I felt to what a degree he had the same quality as his pictures--the quality of looking cleverer than he was.

His wife glanced at him deprecatingly, but his eyes travelled past her to the portrait.

"Mr. Rickham wanted to see it," she began, as if excusing herself. He shrugged his shoulders, still smiling.

"Oh, Rickham found me out long ago," he said lightly; then, passing his arm through mine: "Come and see the rest of the house."

He showed it to me with a kind of naive suburban pride: the bath-rooms, the speaking-tubes, the dress-closets, the trouser-presses--all the complex simplifications of the millionaire's domestic economy. And whenever my wonder paid the expected tribute he said, throwing out his chest a little: "Yes, I really don't see how people manage to live without that."

Well--it was just the end one might have foreseen for him. Only he was, through it all and in spite of it all--as he had been through, and in spite of, his pictures--so handsome, so charming, so disarming, that one longed to cry out: "Be dissatisfied with your leisure!" as once one had longed to say: "Be dissatisfied with your work!"

But, with the cry on my lips, my diagnosis suffered an unexpected check.

"This is my own lair," he said, leading me into a dark plain room at the end of the florid vista. It was square and brown and leathery: no "effects"; no bric-a-brac, none of the air of posing for reproduction in a picture weekly--above all, no least sign of ever having been used as a studio.

The fact brought home to me the absolute finality of Jack's break with his old life.

"Don't you ever dabble with paint any more?" I asked, still looking about for a trace of such activity.

"Never," he said briefly.

"Or water-colour--or etching?"

His confident eyes grew dim, and his cheeks paled a little under their handsome sunburn.

"Never think of it, my dear fellow--any more than if I'd never touched a brush."

And his tone told me in a flash that he never thought of anything else.

I moved away, instinctively embarrassed by my unexpected discovery; and as I turned, my eye fell on a small picture above the mantel-piece--the only object breaking the plain oak panelling of the room.

"Oh, by Jove!" I said.

It was a sketch of a donkey--an old tired donkey, standing in the rain under a wall.

"By Jove--a Stroud!" I cried.

He was silent; but I felt him close behind me, breathing a little quickly.

"What a wonder! Made with a dozen lines--but on everlasting foundations. You lucky chap, where did you get it?"

He answered slowly: "Mrs. Stroud gave it to me."

"Ah--I didn't know you even knew the Strouds. He was such an inflexible hermit."

"I didn't--till after. . . . She sent for me to paint him when he was dead."

"When he was dead? You?"

I must have let a little too much amazement escape through my surprise, for he answered with a deprecating laugh: "Yes--she's an awful simpleton, you know, Mrs. Stroud. Her only idea was to have him done by a fashionable painter--ah, poor Stroud! She thought it the surest way of proclaiming his greatness--of forcing it on a purblind public. And at the moment I was _the_ fashionable painter."

"Ah, poor Stroud--as you say. Was _that_ his history?"

"That was his history. She believed in him, gloried in him--or thought she did. But she couldn't bear not to have all the drawing-rooms with her. She couldn't bear the fact that, on varnishing days, one could always get near enough to see his pictures. Poor woman! She's just a fragment groping for other fragments. Stroud is the only whole I ever knew."

"You ever knew? But you just said--"

Gisburn had a curious smile in his eyes.

"Oh, I knew him, and he knew me--only it happened after he was dead."

I dropped my voice instinctively. "When she sent for you?"

"Yes--quite insensible to the irony. She wanted him vindicated--and by me!"

He laughed again, and threw back his head to look up at the sketch of the donkey. "There were days when I couldn't look at that thing--couldn't face it. But I forced myself to put it here; and now it's cured me--cured me. That's the reason why I don't dabble any more, my dear Rickham; or rather Stroud himself is the reason."

For the first time my idle curiosity about my companion turned into a serious desire to understand him better.

"I wish you'd tell me how it happened," I said.

He stood looking up at the sketch, and twirling between his fingers a cigarette he had forgotten to light. Suddenly he turned toward me.

"I'd rather like to tell you--because I've always suspected you of loathing my work."

I made a deprecating gesture, which he negatived with a good-humoured shrug.

"Oh, I didn't care a straw when I believed in myself--and now it's an added tie between us!"

He laughed slightly, without bitterness, and pushed one of the deep arm-chairs forward. "There: make yourself comfortable--and here are the cigars you like."

He placed them at my elbow and continued to wander up and down the room, stopping now and then beneath the picture.

"How it happened? I can tell you in five minutes--and it didn't take much longer to happen. . . . I can remember now how surprised and pleased I was when I got Mrs. Stroud's note. Of course, deep down, I had always _felt_ there was no one like him--only I had gone with the stream, echoed the usual platitudes about him, till I half got to think he was a failure, one of the kind that are left behind. By Jove, and he _was_ left behind--because he had come to stay! The rest of us had to let ourselves be swept along or go under, but he was high above the current--on everlasting foundations, as you say.

"Well, I went off to the house in my most egregious mood--rather moved, Lord forgive me, at the pathos of poor Stroud's career of failure being crowned by the glory of my painting him! Of course I meant to do the picture for nothing--I told Mrs. Stroud so when she began to stammer something about her poverty. I remember getting off a prodigious phrase about the honour being _mine_--oh, I was princely, my dear Rickham! I was posing to myself like one of my own sitters.

"Then I was taken up and left alone with him. I had sent all my traps in advance, and I had only to set up the easel and get to work. He had been dead only twenty-four hours, and he died suddenly, of heart disease, so that there had been no preliminary work of destruction--his face was clear and untouched. I had met him once or twice, years before, and thought him insignificant and dingy. Now I saw that he was superb.

"I was glad at first, with a merely aesthetic satisfaction: glad to have my hand on such a 'subject.' Then his strange life-likeness began to affect me queerly--as I blocked the head in I felt as if he were watching me do it. The sensation was followed by the thought: if he _were_ watching me, what would he say to my way of working? My strokes began to go a little wild--I felt nervous and uncertain.

"Once, when I looked up, I seemed to see a smile behind his close grayish beard--as if he had the secret, and were amusing himself by holding it back from me. That exasperated me still more. The secret? Why, I had a secret worth twenty of his! I dashed at the canvas furiously, and tried some of my bravura tricks. But they failed me, they crumbled. I saw that he wasn't watching the showy bits--I couldn't distract his attention; he just kept his eyes on the hard passages between. Those were the ones I had always shirked, or covered up with some lying paint. And how he saw through my lies!

"I looked up again, and caught sight of that sketch of the donkey hanging on the wall near his bed. His wife told me afterward it was the last thing he had done--just a note taken with a shaking hand, when he was down in Devonshire recovering from a previous heart attack. Just a note! But it tells his whole history. There are years of patient scornful persistence in every line. A man who had swum with the current could never have learned that mighty up-stream stroke. . . .

"I turned back to my work, and went on groping and muddling; then I looked at the donkey again. I saw that, when Stroud laid in the first stroke, he knew just what the end would be. He had possessed his subject, absorbed it, recreated it. When had I done that with any of my things? They hadn't been born of me--I had just adopted them. . . .

"Hang it, Rickham, with that face watching me I couldn't do another stroke. The plain truth was, I didn't know where to put it--_I had never known_. Only, with my sitters and my public, a showy splash of colour covered up the fact--I just threw paint into their faces. . . . Well, paint was the one medium those dead eyes could see through--see straight to the tottering foundations underneath. Don't you know how, in talking a foreign language, even fluently, one says half the time not what one wants to but what one can? Well--that was the way I painted; and as he lay there and watched me, the thing they called my 'technique' collapsed like a house of cards. He didn't sneer, you understand, poor Stroud--he just lay there quietly watching, and on his lips, through the gray beard, I seemed to hear the question: 'Are you sure you know where you're coming out?'

"If I could have painted that face, with that question on it, I should have done a great thing. The next greatest thing was to see that I couldn't--and that grace was given me. But, oh, at that minute, Rickham, was there anything on earth I wouldn't have given to have Stroud alive before me, and to hear him say: 'It's not too late--I'll show you how'?

"It _was_ too late--it would have been, even if he'd been alive. I packed up my traps, and went down and told Mrs. Stroud. Of course I didn't tell her _that_--it would have been Greek to her. I simply said I couldn't paint him, that I was too moved. She rather liked the idea--she's so romantic! It was that that made her give me the donkey. But she was terribly upset at not getting the portrait--she did so want him 'done' by some one showy! At first I was afraid she wouldn't let me off--and at my wits' end I suggested Grindle. Yes, it was I who started Grindle: I told Mrs. Stroud he was the 'coming' man, and she told somebody else, and so it got to be true. . . . And he painted Stroud without wincing; and she hung the picture among her husband's things. . . ."

He flung himself down in the arm-chair near mine, laid back his head, and clasping his arms beneath it, looked up at the picture above the chimney-piece.

"I like to fancy that Stroud himself would have given it to me, if he'd been able to say what he thought that day."

And, in answer to a question I put half-mechanically--"Begin again?" he flashed out. "When the one thing that brings me anywhere near him is that I knew enough to leave off?"

He stood up and laid his hand on my shoulder with a laugh. "Only the irony of it is that I _am_ still painting--since Grindle's doing it for me! The Strouds stand alone, and happen once--but there's no exterminating our kind of art."""

In [40]:
token = Tokenizer("gpt2")
l = token.encode(text=text)

# Split data: 80% train, 10% validation, 10% test
train_data_tokens = l[:int(0.8*len(l))]
val_data_tokens = l[int(0.8*len(l)):int(0.9*len(l))]
test_data_tokens = l[int(0.9*len(l)):]

print(f"Total tokens: {len(l)}")
print(f"Train tokens: {len(train_data_tokens)}")
print(f"Val tokens: {len(val_data_tokens)}")
print(f"Test tokens: {len(test_data_tokens)}")

# Create datasets
train_dataset = GPTDataset(train_data_tokens, block_size=4, stride=3)
val_dataset = GPTDataset(val_data_tokens, block_size=4, stride=3)
test_dataset = GPTDataset(test_data_tokens, block_size=4, stride=3)

Total tokens: 5145
Train tokens: 4116
Val tokens: 514
Test tokens: 515


In [41]:
from torch.utils.data.dataloader import DataLoader

# Create dataloaders for train, validation, and test
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=6,
    shuffle=True
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=6,
    shuffle=False
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=6,
    shuffle=False
)

In [42]:
# Check dataloaders
print("\nTrain loader:")
for batch in train_loader:
    print("Input (x) shape:", batch[0].shape)
    print("Target (y) shape:", batch[1].shape)
    break

print("\nValidation loader:")
for batch in val_loader:
    print("Input (x) shape:", batch[0].shape)
    print("Target (y) shape:", batch[1].shape)
    break

print("\nTest loader:")
for batch in test_loader:
    print("Input (x) shape:", batch[0].shape)
    print("Target (y) shape:", batch[1].shape)
    break


Train loader:
Input (x) shape: torch.Size([6, 4])
Target (y) shape: torch.Size([6, 4])

Validation loader:
Input (x) shape: torch.Size([6, 4])
Target (y) shape: torch.Size([6, 4])

Test loader:
Input (x) shape: torch.Size([6, 4])
Target (y) shape: torch.Size([6, 4])


In [43]:
# Training function with validation
def train_model(model, train_loader, val_loader, epochs, learning_rate, device='cpu'):
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    
    train_losses = []
    val_losses = []
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        total_train_loss = 0
        for batch_idx, (x, y) in enumerate(train_loader):
            x, y = x.to(device), y.to(device)
            
            # Forward pass
            logits = model(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1))
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
            optimizer.step()
            
            total_train_loss += loss.item()
            
            if (batch_idx + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs}, Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")
        
        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        
        # Validation phase
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                logits = model(x)
                loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1))
                total_val_loss += loss.item()
        
        avg_val_loss = total_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        
        print(f"\nEpoch {epoch+1}/{epochs} completed.")
        print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}\n")
    
    return model, train_losses, val_losses

In [44]:
# Train the model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Start training
trained_model, train_losses, val_losses = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=3,
    learning_rate=5e-5,
    device=device
)

Using device: cpu
Epoch 1/3, Batch 10/229, Loss: 4.9383
Epoch 1/3, Batch 10/229, Loss: 4.9383
Epoch 1/3, Batch 20/229, Loss: 4.4576
Epoch 1/3, Batch 20/229, Loss: 4.4576
Epoch 1/3, Batch 30/229, Loss: 5.5033
Epoch 1/3, Batch 30/229, Loss: 5.5033
Epoch 1/3, Batch 40/229, Loss: 5.5769
Epoch 1/3, Batch 40/229, Loss: 5.5769
Epoch 1/3, Batch 50/229, Loss: 4.4779
Epoch 1/3, Batch 50/229, Loss: 4.4779
Epoch 1/3, Batch 60/229, Loss: 3.7250
Epoch 1/3, Batch 60/229, Loss: 3.7250
Epoch 1/3, Batch 70/229, Loss: 4.9412
Epoch 1/3, Batch 70/229, Loss: 4.9412
Epoch 1/3, Batch 80/229, Loss: 4.9408
Epoch 1/3, Batch 80/229, Loss: 4.9408
Epoch 1/3, Batch 90/229, Loss: 5.2071
Epoch 1/3, Batch 90/229, Loss: 5.2071
Epoch 1/3, Batch 100/229, Loss: 3.9306
Epoch 1/3, Batch 100/229, Loss: 3.9306
Epoch 1/3, Batch 110/229, Loss: 4.7494
Epoch 1/3, Batch 110/229, Loss: 4.7494
Epoch 1/3, Batch 120/229, Loss: 3.8151
Epoch 1/3, Batch 120/229, Loss: 3.8151
Epoch 1/3, Batch 130/229, Loss: 3.2278
Epoch 1/3, Batch 130/229,

In [45]:
# Evaluate on test set
device = 'cuda' if torch.cuda.is_available() else 'cpu'
trained_model.eval()
total_test_loss = 0

with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        logits = trained_model(x)
        loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1))
        total_test_loss += loss.item()

avg_test_loss = total_test_loss / len(test_loader)
print(f"Test Loss: {avg_test_loss:.4f}")
print(f"Test Perplexity: {torch.exp(torch.tensor(avg_test_loss)):.2f}")

# Generate text with fine-tuned model
print("\n" + "="*50)
print("GENERATING TEXT")
print("="*50)

test_prompt = "I HAD always thought"
tokenizer = Tokenizer("gpt2")
test_ids = torch.tensor([tokenizer.encode(test_prompt)], dtype=torch.long).to(device)

generated = generate_text_simple(
    trained_model, 
    test_ids, 
    max_new_tokens=50, 
    context_size=config["context_length"], 
    temperature=0.8
)

generated_text = tokenizer.decode(generated[0].tolist())
print("\nGenerated text:")
print(generated_text)

Test Loss: 4.8769
Test Perplexity: 131.22

GENERATING TEXT

Generated text:
I HAD always thought that he was rich; and now I see that he is poor."

He laughed again, and went off to his own house, where he stood alone in the sunlit terrace, surrounded by his handsome wife, and his elegant brown-

Generated text:
I HAD always thought that he was rich; and now I see that he is poor."

He laughed again, and went off to his own house, where he stood alone in the sunlit terrace, surrounded by his handsome wife, and his elegant brown-


In [46]:
# Save the fine-tuned model
torch.save({
    'model_state_dict': trained_model.state_dict(),
    'config': config,
}, 'finetuned_gpt2.pt')
print("Model saved!")

Model saved!


In [47]:
# Optional: Load the saved model later
# def load_finetuned_model(checkpoint_path, config):
#     model = Llm("gpt2", config)
#     checkpoint = torch.load(checkpoint_path)
#     model.load_state_dict(checkpoint['model_state_dict'])
#     return model

# Example usage:
# loaded_model = load_finetuned_model('finetuned_gpt2.pt', config)